In [3]:
import os

import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer

In [4]:
# import plotting libraries
import matplotlib
import matplotlib.pyplot as plt

%matplotlib inline
plt.style.use('seaborn-v0_8') # pretty matplotlib plots

import seaborn as sns
sns.set_theme('notebook', style='whitegrid', font_scale=1.25)

# autoload changes in other files, so you don't have to restart the Jupyter kernel each time you make a change to the imported code.
%load_ext autoreload
%autoreload 2

In [5]:
from load_train_data import overview_data
DATA_PATH = './data_reviews'

x_train_nf = pd.read_csv(os.path.join(DATA_PATH, 'x_train.csv'))
y_train_n = pd.read_csv(os.path.join(DATA_PATH, 'y_train.csv'))

overview_data(x_train_nf, y_train_n)

Shape of x_train_df: (2400, 2)
Shape of y_train_df: (2400, 1)

Print three instance from the training set in lists:
Input (website, review):
[['amazon', 'Oh and I forgot to also mention the weird color effect it has on your phone.'], ['amazon', "THAT one didn't work either."], ['amazon', 'Waste of 13 bucks.']]
Label:
[0, 0, 0]


More data from training set:
row     0 | y = 0 | Oh and I forgot to also mention the weird color effect it has on your phone.
row     1 | y = 0 | THAT one didn't work either.
row     2 | y = 0 | Waste of 13 bucks.
row     3 | y = 0 | Product is useless, since it does not have enough charging current to charge the 2 cellphones I was planning to use it with.
row     4 | y = 0 | None of the three sizes they sent with the headset would stay in my ears.
...
row  2395 | y = 1 | The sweet potato fries were very nice and seasoned well.
row  2396 | y = 1 | I could eat their bruschetta all day it is devine.
row  2397 | y = 1 | Ambience is perfect.
row  2398 | y = 1 | We 

In [6]:
x_train_nf.loc[:, 'text']

0       Oh and I forgot to also mention the weird colo...
1                            THAT one didn't work either.
2                                      Waste of 13 bucks.
3       Product is useless, since it does not have eno...
4       None of the three sizes they sent with the hea...
                              ...                        
2395    The sweet potato fries were very nice and seas...
2396    I could eat their bruschetta all day it is dev...
2397                                 Ambience is perfect.
2398    We ordered the duck rare and it was pink and t...
2399         Service was nice and the company was better!
Name: text, Length: 2400, dtype: object

In [7]:
import sklearn.linear_model
from sklearn.feature_extraction.text import CountVectorizer

In [8]:
vectorizer = CountVectorizer()
corpus = x_train_nf.loc[:, 'text']
input_vectors_nv = vectorizer.fit_transform(corpus)
input_vectors_nv.shape

(2400, 4510)

In [24]:
input_vectors_nv[0]

(1, 4510)